In [19]:
import cv2 as cv, os, numpy as np

import skimage
from skimage import measure

# Contrast Enhancement

DIP Technique 1

In [15]:
def contrast_enhancement(img) :
    
    equ_img = cv.equalizeHist(img) # Applying Histogram Equalization
    
    clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    clahe_img = clahe.apply(equ_img) # Applying contrast enhancement
    
    return equ_img

In [16]:
images_dir_path = r"C:\Local Disk\Programming\Infratech project\Image Processing\DIP Technique 1 - Contrast Enhancement\test images"

for img_file_name in os.listdir(images_dir_path) :
    print(img_file_name)
    img = cv.imread(os.path.join(images_dir_path,img_file_name),0)
    img = contrast_enhancement(img)
    cv.imwrite(os.path.join(images_dir_path,img_file_name[:-4]+'_enhanced.jpg'),img)

Day 1 Grey.jpg
Day 1.jpg
Day 2 Grey.jpg
Day 2.jpg
Day 3 Grey.jpg
Day 3.jpg
Night 1 Grey.jpg
Night 1.jpg
Night 2 Grey.jpg
Night 2.jpg
Night 3 Grey.jpg
Night 3.jpg
Night 4.jpg


# Glare Reduction

In [21]:
def glare_reduction(image) :
    
    gray = cv.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    image = image.astype(np.float32)/255
    
    blur_img = cv.GaussianBlur(gray,(9,9),0)
    _, thresh_img = cv2.threshold(blur_img, 200, 255, cv.THRESH_BINARY)
    
    labels = measure.label( thresh_img, connectivity=2, background=0 )
    mask = np.zeros( thresh_img.shape, dtype="uint8" )

    for label in np.unique( labels ):
        # if this is the background label, ignore it
        if label == 0:
            continue
            
        # otherwise, construct the label mask and count the
        # number of pixels
        labelMask = np.zeros( thresh_img.shape, dtype="uint8" )
        labelMask[labels == label] = 255
        numPixels = cv2.countNonZero( labelMask )
        
        # if the number of pixels in the component is sufficiently
        # large, then add it to our mask of "large blobs"
        if numPixels > 300:
            mask = cv2.add( mask, labelMask )
            
    
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)     
    #print(contours)
    #print(hierarchy)

    cv2.drawContours(image, contours, -1, (0, 255, 0), 2)
    
    gray1 = gray

    for contour in contours:
        x,y,w,h = cv2.boundingRect(contour)
        image = cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
        #print("width: " , w)
        #print("height: ", h)

        roi = gray[y:(y+h), x:(x+w)]
        #roi = cv2.resize(roi, (640, 480))
        #roi = cv2.cvtColor(roi, cv2.COLOR_RGB2GRAY)
        #cv2.imshow("roi", roi)
        #print(roi)


        alpha = 0.7 # Contrast control (1.0-3.0)
        beta = 45 # Brightness control (0-100)

        contrast_adjust = cv2.convertScaleAbs(roi, alpha=alpha, beta=beta)
        #contrast_adjust = cv2.convertScaleAbs(roi)

        denoised_img = cv2.fastNlMeansDenoising(contrast_adjust,None,10,7,21)

        #cv2.imshow("contrast", contrast_adjust)
        #print(contrast_adjust)
        #cv2.imshow("denoised_img", denoised_img)


        gray = gray.copy()
        gray1 = gray
        gray[y: y + h, x: x + w] = contrast_adjust
        gray1[y: y + h, x: x + w] = denoised_img # important for us
        
    return gray1

In [22]:
images_dir_path = r"C:\Local Disk\Programming\Infratech project\Image Processing\DIP Technique 2 - Glare Reduction\test images"

for img_file_name in os.listdir(images_dir_path) :
    print(img_file_name)
    img = cv.imread(os.path.join(images_dir_path,img_file_name))
    img = glare_reduction(img)
    cv.imwrite(os.path.join(images_dir_path,img_file_name[:-4]+'_enhanced.jpg'),img)

Day 1 Grey.jpg
Day 1.jpg
Day 2 Grey.jpg
Day 2.jpg
Day 3 Grey.jpg
Day 3.jpg
Night 1 Grey.jpg
Night 1.jpg
Night 2 Grey.jpg
Night 2.jpg
Night 3 Grey.jpg
Night 3.jpg
Night 4.jpg
